In [1]:
import pyspark


In [2]:
df = sqlContext.sql("SELECT * FROM sales_info_csv WHERE Sales is not null ")

In [3]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam| 200|
 GOOG|Charlie| 120|
 GOOG| Frank| 340|
 MSFT| Tina| 600|
 MSFT| Amy| 124|
 MSFT|Vanessa| 243|
 FB| Carl| 870|
 FB| Sarah| 350|
 APPL| John| 250|
+-------+-------+-----+

In [4]:
#DATAFRAMES BASIC

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [7]:
df = spark.read.json('FileStore/tables/people.json')

In [8]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [9]:
df.printSchema()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [10]:
df.columns

Out[ 10 ]: ['age', 'name']

In [11]:
df.describe().show()

+-------+------------------+-------+
summary| age| name|
+-------+------------------+-------+
 count| 2| 3|
 mean| 24.5| null|
 stddev|7.7781745930520225| null|
 min| 19| Andy|
 max| 30|Michael|
+-------+------------------+-------+

In [12]:
from pyspark.sql.types import (StructField,
                               StringType,
                               IntegerType,
                               StructType)

In [13]:
data_schema = [StructField('age',IntegerType(),True),
              StructField('name',StringType(),True)]

In [14]:
final_struc = StructType(fields=data_schema)

In [15]:
df1 = spark.read.json('FileStore/tables/people.json',schema=final_struc)

In [16]:
df1.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [17]:
df1.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('aggs').getOrCreate()

df = spark.read.csv('FileStore/tables/sales_info.csv',inferSchema=True,header=True)

In [19]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [20]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [21]:
df.groupBy('Company').count().show()

+-------+-----+
Company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [22]:
df.groupBy('Company').mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [23]:
df.agg({'Sales':'sum'}).show()

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

In [24]:
df.agg({'Sales':'min'}).show()

+----------+
min(Sales)|
+----------+
 120.0|
+----------+

In [25]:
group_data = df.groupBy('Company')
group_data.mean().show()
group_data.agg({'Sales':'max'}).show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [26]:
from pyspark.sql.functions import countDistinct,avg,stddev
df.select(countDistinct('Company')).show()

+-----------------------+
count(DISTINCT Company)|
+-----------------------+
 4|
+-----------------------+

In [27]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
 Average Sales|
+-----------------+
360.5833333333333|
+-----------------+

In [28]:
from pyspark.sql.functions import format_number
std = df.select(stddev('Sales').alias('Sales_stddev'))
std.show()
std.select(format_number('Sales_stddev',2).alias('std')).show()


+------------------+
 Sales_stddev|
+------------------+
250.08742410799007|
+------------------+

+------+
 std|
+------+
250.09|
+------+

In [29]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [30]:
df.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [31]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+

In [32]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Project').getOrCreate()
df_walmart = spark.read.csv('FileStore/tables/walmart_stock.csv',header=True,inferSchema=True)
df_walmart.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
2012-01-03 00:00:00| 59.970001| 61.060001| 59.869999| 60.330002|12668800|52.619234999999996|
2012-01-04 00:00:00|60.209998999999996| 60.349998| 59.470001|59.709998999999996| 9593300| 52.078475|
2012-01-05 00:00:00| 59.349998| 59.619999| 58.369999| 59.419998|12768200| 51.825539|
2012-01-06 00:00:00| 59.419998| 59.450001| 58.869999| 59.0| 8069400| 51.45922|
2012-01-09 00:00:00| 59.029999| 59.549999| 58.919998| 59.18| 6679300|51.616215000000004|
2012-01-10 00:00:00| 59.43|59.709998999999996| 58.98|59.040001000000004| 6907300| 51.494109|
2012-01-11 00:00:00| 59.060001| 59.529999|59.040001000000004| 59.400002| 6365600| 51.808098|
2012-01-12 00:00:00|59.790001000000004| 60.0| 59.400002| 59.5| 7236400|51.895315999999994|
2012-01-13 00:00:00| 59.18|59.610001000000004|59.009997999999996|59.540001000000004| 7729300|51.930203999999996|
2012-01-17 00:00:00| 59.869999|60.110001000000004| 59.52| 59.849998| 8500000| 52.200581|
2012-01-18 00:00:00|59.790001000000004| 60.029999| 59.650002|60.009997999999996| 5911400| 52.340131|
2012-01-19 00:00:00| 59.93| 60.73| 59.75|60.610001000000004| 9234600| 52.863447|
2012-01-20 00:00:00| 60.75| 61.25| 60.669998|61.009997999999996|10378800|53.212320999999996|
2012-01-23 00:00:00| 60.810001| 60.98|60.509997999999996| 60.91| 7134100| 53.125104|
2012-01-24 00:00:00| 60.75| 62.0| 60.75|61.389998999999996| 7362800| 53.54375400000001|
2012-01-25 00:00:00| 61.18|61.610001000000004|61.040001000000004| 61.470001| 5915800| 53.61353100000001|
2012-01-26 00:00:00| 61.799999| 61.84| 60.77| 60.970001| 7436200| 53.177436|
2012-01-27 00:00:00|60.860001000000004| 61.119999|60.540001000000004|60.709998999999996| 6287300| 52.950665|
2012-01-30 00:00:00| 60.470001| 61.32| 60.349998| 61.299999| 7636900|53.465256999999994|
2012-01-31 00:00:00| 61.529999| 61.57| 60.580002|61.360001000000004| 9761500|53.517590000000006|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
only showing top 20 rows

In [33]:
#What are the column names
df_walmart.columns

Out[ 37 ]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [34]:
#What does the Schema look like?¶
df_walmart.printSchema()

root
-- Date: timestamp (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [35]:
df_walmart.schema


Out[ 44 ]: StructType(List(StructField(Date,TimestampType,true),StructField(Open,DoubleType,true),StructField(High,DoubleType,true),StructField(Low,DoubleType,true),StructField(Close,DoubleType,true),StructField(Volume,IntegerType,true),StructField(Adj Close,DoubleType,true)))

In [36]:
#Print out the first 5 columns

for items in df_walmart.head(5):
  print(items)
  print('\n')

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)

In [37]:
#Use describe() to learn about the DataFrame
#like pandas Dataframe .describe()
df_walmart.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [38]:
df_walmart.describe().printSchema()

root
-- summary: string (nullable = true)
-- Open: string (nullable = true)
-- High: string (nullable = true)
-- Low: string (nullable = true)
-- Close: string (nullable = true)
-- Volume: string (nullable = true)
-- Adj Close: string (nullable = true)

In [39]:
result = df_walmart.describe()

In [40]:
result.show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [41]:
from pyspark.sql.functions import format_number
result.select(result['summary'],
              format_number(result['open'].cast('float'),2).alias('open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
summary| open| High| Low| Close| Volume|
+-------+--------+--------+--------+--------+--------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1258|
 mean| 72.36| 72.84| 71.92| 72.39| 8222093|
 stddev| 6.77| 6.77| 6.74| 6.76| 4519780|
 min| 56.39| 57.06| 56.30| 56.42| 2094900|
 max| 90.80| 90.97| 89.25| 90.47|80898100|
+-------+--------+--------+--------+--------+--------+

In [42]:
#Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day
df1 = df_walmart.withColumn("HV Ratio",df_walmart['High']/df_walmart['Volume'])
df1.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
 Date| Open| High| Low| Close| Volume| Adj Close| HV Ratio|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
2012-01-03 00:00:00| 59.970001| 61.060001| 59.869999| 60.330002|12668800|52.619234999999996|4.819714653321546E-6|
2012-01-04 00:00:00|60.209998999999996| 60.349998| 59.470001|59.709998999999996| 9593300| 52.078475|6.290848613094555E-6|
2012-01-05 00:00:00| 59.349998| 59.619999| 58.369999| 59.419998|12768200| 51.825539|4.669412994783916E-6|
2012-01-06 00:00:00| 59.419998| 59.450001| 58.869999| 59.0| 8069400| 51.45922|7.367338463826307E-6|
2012-01-09 00:00:00| 59.029999| 59.549999| 58.919998| 59.18| 6679300|51.616215000000004|8.915604778943901E-6|
2012-01-10 00:00:00| 59.43|59.709998999999996| 58.98|59.040001000000004| 6907300| 51.494109|8.644477436914568E-6|
2012-01-11 00:00:00| 59.060001| 59.529999|59.040001000000004| 59.400002| 6365600| 51.808098|9.351828421515645E-6|
2012-01-12 00:00:00|59.790001000000004| 60.0| 59.400002| 59.5| 7236400|51.895315999999994| 8.29141562102703E-6|
2012-01-13 00:00:00| 59.18|59.610001000000004|59.009997999999996|59.540001000000004| 7729300|51.930203999999996|7.712212102001476E-6|
2012-01-17 00:00:00| 59.869999|60.110001000000004| 59.52| 59.849998| 8500000| 52.200581|7.071764823529412E-6|
2012-01-18 00:00:00|59.790001000000004| 60.029999| 59.650002|60.009997999999996| 5911400| 52.340131|1.015495466386981E-5|
2012-01-19 00:00:00| 59.93| 60.73| 59.75|60.610001000000004| 9234600| 52.863447|6.576354146362592...|
2012-01-20 00:00:00| 60.75| 61.25| 60.669998|61.009997999999996|10378800|53.212320999999996| 5.90145296180676E-6|
2012-01-23 00:00:00| 60.810001| 60.98|60.509997999999996| 60.91| 7134100| 53.125104|8.547679455011844E-6|
2012-01-24 00:00:00| 60.75| 62.0| 60.75|61.389998999999996| 7362800| 53.54375400000001|8.420709512685392E-6|
2012-01-25 00:00:00| 61.18|61.610001000000004|61.040001000000004| 61.470001| 5915800| 53.61353100000001|1.041448341728929...|
2012-01-26 00:00:00| 61.799999| 61.84| 60.77| 60.970001| 7436200| 53.177436|8.316075414862431E-6|
2012-01-27 00:00:00|60.860001000000004| 61.119999|60.540001000000004|60.709998999999996| 6287300| 52.950665|9.721183814992126E-6|
2012-01-30 00:00:00| 60.470001| 61.32| 60.349998| 61.299999| 7636900|53.465256999999994|8.029436027707578E-6|
2012-01-31 00:00:00| 61.529999| 61.57| 60.580002|61.360001000000004| 9761500|53.517590000000006|6.307432259386365E-6|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
only showing top 20 rows

In [43]:
df1.select('HV Ratio').show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

In [44]:
#What day had the Peak High in Price?
df_walmart.orderBy(df_walmart['High'].desc()).head()[0]

Out[ 70 ]: datetime.datetime(2015, 1, 13, 0, 0)

In [45]:
#What is the mean of the Close column?¶
## deribe from df_wlamart.describe()
result.select('Close',result['summary'] =='mean').show()

+-----------------+----------------+
 Close|(summary = mean)|
+-----------------+----------------+
 1258| false|
72.38844998012726| true|
6.756859163732991| false|
 56.419998| false|
 90.470001| false|
+-----------------+----------------+

In [46]:
df2 = result[result['summary']=='mean']
df2.select('Close').show()

+-----------------+
 Close|
+-----------------+
72.38844998012726|
+-----------------+

In [47]:
from pyspark.sql.functions import mean
df_walmart.select(mean('Close')).show()

+-----------------+
 avg(Close)|
+-----------------+
72.38844998012726|
+-----------------+

In [48]:
#What is the max and min of the Volume column?
from pyspark.sql.functions import max,min
df_walmart.select(min('Volume').alias('min_volume'),max('Volume').alias('max_volume')).show()


+----------+----------+
min_volume|max_volume|
+----------+----------+
 2094900| 80898100|
+----------+----------+

In [49]:
#How many days was the Close lower than 60 dollars?¶
df_walmart.select('Date','Close','Volume').filter(df_walmart['Close'] < 60).show()

+-------------------+------------------+--------+
 Date| Close| Volume|
+-------------------+------------------+--------+
2012-01-04 00:00:00|59.709998999999996| 9593300|
2012-01-05 00:00:00| 59.419998|12768200|
2012-01-06 00:00:00| 59.0| 8069400|
2012-01-09 00:00:00| 59.18| 6679300|
2012-01-10 00:00:00|59.040001000000004| 6907300|
2012-01-11 00:00:00| 59.400002| 6365600|
2012-01-12 00:00:00| 59.5| 7236400|
2012-01-13 00:00:00|59.540001000000004| 7729300|
2012-01-17 00:00:00| 59.849998| 8500000|
2012-02-22 00:00:00| 58.599998|28630200|
2012-02-23 00:00:00|58.540001000000004|14880300|
2012-02-24 00:00:00|58.790001000000004| 9925900|
2012-02-27 00:00:00|58.459998999999996|12258800|
2012-02-28 00:00:00| 58.93|10761900|
2012-02-29 00:00:00| 59.080002|11484400|
2012-03-01 00:00:00| 58.82|16283900|
2012-03-02 00:00:00|59.009997999999996| 9848100|
2012-03-05 00:00:00| 59.400002| 9651000|
2012-03-06 00:00:00| 58.970001| 9057100|
2012-03-07 00:00:00|59.860001000000004|14916900|
+-------------------+------------------+--------+
only showing top 20 rows

In [50]:
df_walmart.filter(df_walmart['Close'] < 60).count()

Out[ 98 ]: 81

In [51]:
#What percentage of the time was the High greater than 80 dollars ?¶
cnt80 =df_walmart.filter(df_walmart['High'] > 80).count() 
cnt = df_walmart.select('High').count()
percent80 = (cnt80 * 1.0/ cnt) * 100
print('percentage of the time was the High greater than 80 dollars is : {}'.format(percent80 ))

percentage of the time was the High greater than 80 dollars is : 9.14149443561

In [52]:
#What is the Pearson correlation between High and Volume?
from pyspark.sql.functions import corr
df_walmart.select(corr('High','Volume').alias('correlation')).show()

+-------------------+
 correlation|
+-------------------+
-0.3384326061737161|
+-------------------+

In [53]:
#What is the max High per year?
from pyspark.sql.functions import year
df3 = df_walmart.withColumn('Year',year(df_walmart['Date']))
df_high_year  = df3.groupBy('Year').max()
df_high_year.show()

+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
Year| max(Open)|max(High)| max(Low)|max(Close)|max(Volume)| max(Adj Close)|max(Year)|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
2015| 90.800003|90.970001| 89.25| 90.470001| 80898100|84.91421600000001| 2015|
2013| 81.209999|81.370003| 80.82| 81.209999| 25683700| 73.929868| 2013|
2014|87.08000200000001|88.089996|86.480003| 87.540001| 22812400|81.70768000000001| 2014|
2012| 77.599998|77.599998|76.690002| 77.150002| 38007300| 68.568371| 2012|
2016| 74.5|75.190002|73.629997| 74.300003| 35076700| 73.233524| 2016|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+

In [54]:
#What is the average Close for each Calendar Month?
#In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.
from pyspark.sql.functions import month
df_month = df_walmart.withColumn('Month',month(df_walmart['Date']))
df_avg_month = df_month.orderBy('Month').groupBy('Month').avg()
df_avg_month.select('Month','avg(Close)').show()

+-----+-----------------+
Month| avg(Close)|
+-----+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+-----+-----------------+